# IBM Data Science Professional Certificate capstone project
This notebook contains work towards the Professional Certificate capstone project

In [1]:
import pandas as pd
import numpy as np

In [2]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!
